In [2]:
import sys
sys.path.append('../')

In [14]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
%reload_ext autoreload

In [4]:
from scripts.detect_voids import get_neightbours, search_voids_bb_neightbours
from packages.yolo_predict import YOLO_Pred
from concurrent.futures import ThreadPoolExecutor
import pandas as pd
import cv2
import multiprocessing
from functools import partial
import itertools

In [5]:
yolo = YOLO_Pred('/home/cremerf/FinalProject/data/first_training/weights/bestnoft.onnx', '/home/cremerf/FinalProject/data/config_blmodel.yaml')

In [7]:
image = cv2.imread('/home/cremerf/FinalProject/eudes-fede/test_imgs/test_7.jpg')

In [8]:
df_predictions = yolo.predictions(image)

In [13]:
def bb_intersection_over_union(df_predictions: pd.DataFrame, index_key:int, index_neightbour:int) -> float:

    a = index_key
    xA1 = df_predictions.loc[a][0] - df_predictions.loc[a][2]/2
    yA1 = df_predictions.loc[a][1] + df_predictions.loc[a][3]/2
    xA2 = df_predictions.loc[a][0] + df_predictions.loc[a][2]/2
    yA2 = df_predictions.loc[a][1] - df_predictions.loc[a][3]/2
    boxA = [xA1, yA1, xA2, yA2 ]

    b = index_neightbour # indices vecinos detectados que estan en el diccionario
    xB1 = df_predictions.loc[b][0] - df_predictions.loc[b][2]/2
    yB1 = df_predictions.loc[b][1] + df_predictions.loc[b][3]/2
    xB2 = df_predictions.loc[b][0] + df_predictions.loc[b][2]/2
    yB2 = df_predictions.loc[b][1] - df_predictions.loc[b][3]/2
    boxB = [xB1, yB1,xB2, yB2 ]
    
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])


    # compute the area of intersection rectangle
    interArea = (xB - xA) * (yB - yA)

    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)

    # return the intersection over union value
    return iou

In [ ]:
# para left w = -w y h = 0
# para right w = +w y h = 0
# para top w = 0 y h = + h

In [47]:
image = cv2.imread(filename='/home/cremerf/FinalProject/eudes-fede/test_imgs/test_7.jpg')

In [18]:
image.shape[0:2]

(3264, 2448)

In [130]:
df_predictions.loc[1059]

Center_X    210.405991
Center_Y    233.077942
Width        24.668375
Height       70.583984
Name: 1059, dtype: float64

In [129]:
df_predictions.loc[1059][2]

24.66837501525879

In [123]:
xA1 = df_predictions.loc[1059][0] - df_predictions.loc[1059][2]/2 - w_index_a
yA1 = df_predictions.loc[1059][1] - df_predictions.loc[1059][3]/2
xA2 = df_predictions.loc[1059][0] + df_predictions.loc[1059][2]/2 - w_index_a
yA2 = df_predictions.loc[1059][1] + df_predictions.loc[1059][3]/2

In [125]:
xA1, yA1, xA2, yA2

(184.86670780181885,
 268.36993408203125,
 209.53508281707764,
 197.78594970703125)

In [103]:
def search_voids_bb_neightbours_left(df_predictions: pd.DataFrame, dict_of_neightbours: dict):
    
    list_of_voids = []
    for index_a, index_b in dict_of_neightbours.items():

        first_lists = []
        h_image, w_image = image.shape[0:2] # obtengo limites de la imagen
        w_index_a = df_predictions.loc[int(index_a)][2]
        h_index_a = df_predictions.loc[int(index_a)][3]
        
        xA1 = df_predictions.loc[int(index_a)][0] - df_predictions.loc[int(index_a)][2]/2 - w_index_a
        yA1 = df_predictions.loc[int(index_a)][1] - df_predictions.loc[int(index_a)][3]/2
        xA2 = df_predictions.loc[int(index_a)][0] + df_predictions.loc[int(index_a)][2]/2 - w_index_a
        yA2 = df_predictions.loc[int(index_a)][1] + df_predictions.loc[int(index_a)][3]/2
        boxA = [xA1, yA1, xA2, yA2]

        void_number = 1
        void_text = f'Void number {void_number}'        
        X_center_A = df_predictions.loc[int(index_a)][0] - df_predictions.loc[int(index_a)][2]  #X_center - Width
        Y_center_A = df_predictions.loc[int(index_a)][1]  - df_predictions.loc[int(index_a)][2]  #Y_center - Width

        print(f'Evaluating {index_a}...')
        if 0 < xA1 < w_image and 0 < xA2 < w_image and 0 < yA1 < h_image and  0 < yA2 < h_image:
                trigger = True
                for item in index_b:
                    xB1 = df_predictions.loc[item][0] - df_predictions.loc[item][2]/2
                    yB1 = df_predictions.loc[item][1] - df_predictions.loc[item][3]/2
                    xB2 = df_predictions.loc[item][0] + df_predictions.loc[item][2]/2
                    yB2 = df_predictions.loc[item][1] + df_predictions.loc[item][3]/2
                    boxB = [xB1, yB1,xB2, yB2]

                    xA = max(boxA[0], boxB[0])
                    yA = max(boxA[1], boxB[1])
                    xB = min(boxA[2], boxB[2])
                    yB = min(boxA[3], boxB[3])

                    interArea = (xB - xA) * (yB - yA)

                    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
                    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])

                    iou = interArea / float(boxAArea + boxBArea - interArea)
                    trigger = trigger and (iou < 0.1)
                #print(f'Trigger: {trigger}')
                if trigger == False:
                    pass
                    #print(f'No hay espacio vacio a la izquierda de {index_a}')
                else:
                    print(f'A la izquierda de {index_a} hay espacio vacio')
                    
                    list_of_voids.append(boxA)

    return list_of_voids

In [ ]:
# izquierda (-1)
xA1 = df_predictions.loc[int(index_a)][0] - df_predictions.loc[int(index_a)][2]/2 - k * w_index_a
yA1 = df_predictions.loc[int(index_a)][1] - df_predictions.loc[int(index_a)][3]/2
xA2 = df_predictions.loc[int(index_a)][0] + df_predictions.loc[int(index_a)][2]/2 - k * w_index_a
yA2 = df_predictions.loc[int(index_a)][1] + df_predictions.loc[int(index_a)][3]/2

In [ ]:
# derecha (+1)
xA1 = df_predictions.loc[int(index_a)][0] - df_predictions.loc[int(index_a)][2]/2 + k * w_index_a
yA1 = df_predictions.loc[int(index_a)][1] - df_predictions.loc[int(index_a)][3]/2
xA2 = df_predictions.loc[int(index_a)][0] + df_predictions.loc[int(index_a)][2]/2 + k * w_index_a
yA2 = df_predictions.loc[int(index_a)][1] + df_predictions.loc[int(index_a)][3]/2

In [ ]:
# arriba 
xA1 = df_predictions.loc[int(index_a)][0] - df_predictions.loc[int(index_a)][2]/2 
yA1 = df_predictions.loc[int(index_a)][1] - df_predictions.loc[int(index_a)][3]/2 - h_index_a
xA2 = df_predictions.loc[int(index_a)][0] + df_predictions.loc[int(index_a)][2]/2 
yA2 = df_predictions.loc[int(index_a)][1] + df_predictions.loc[int(index_a)][3]/2 - h_index_a

In [ ]:
# arriba 
xA1 = df_predictions.loc[int(index_a)][0] - df_predictions.loc[int(index_a)][2]/2 + k * w_index_a # Para izquierda: (-1) / Para derecha: (1) / Para arriba: 0
yA1 = df_predictions.loc[int(index_a)][1] - df_predictions.loc[int(index_a)][3]/2 + k * h_index_a # Para izquierda: 0 / Para derecha: 0 / Para arriba: -1
xA2 = df_predictions.loc[int(index_a)][0] + df_predictions.loc[int(index_a)][2]/2 + k * w_index_a # Para izquierda: (-1) / Para derecha: (1) / Para arriba: 0
yA2 = df_predictions.loc[int(index_a)][1] + df_predictions.loc[int(index_a)][3]/2 + K * h_index_a # Para izquierda: 0 / Para derecha: 0 / Para arriba: -1

In [11]:
img_path = '/home/cremerf/FinalProject/eudes-fede/test_imgs/test_7.jpg'

In [14]:
list_of_dicts = get_neightbours(img_path= img_path, df_predictions= df_predictions)

In [241]:
X_center_0 = df_predictions.loc[572][0]
Y_center_0 = df_predictions.loc[572][1]
INPUT_WH_YOLO = 640

In [232]:
X_center_0 + 160

381.5714416503906

In [221]:
filter_1 = (df_predictions.loc[:, 'X_center'] > (X_center_0 - INPUT_WH_YOLO/4))

In [222]:
filter_2 = ((df_predictions.loc[:, 'X_center'] < (X_center_0 + INPUT_WH_YOLO/4)))

In [223]:
df_predictions_pre = df_predictions[filter_1]

In [225]:
df_predictions_last =  df_predictions[filter_2]

In [242]:
df_predictions_final = df_predictions[(df_predictions.loc[:, 'X_center'] < (X_center_0 + INPUT_WH_YOLO/8)) & (df_predictions.loc[:, 'X_center'] > (X_center_0 - INPUT_WH_YOLO/8)) & (df_predictions.loc[:, 'Y_center'] > (Y_center_0 - INPUT_WH_YOLO/8)) & (df_predictions.loc[:, 'Y_center'] < (Y_center_0 + INPUT_WH_YOLO/8))]

In [245]:
filter_1 = (df_predictions.loc[:, 'X_center'] < (X_center_0 + INPUT_WH_YOLO/8))
filter_2 = (df_predictions.loc[:, 'X_center'] > (X_center_0 - INPUT_WH_YOLO/8))
filter_3 = (df_predictions.loc[:, 'Y_center'] > (Y_center_0 - INPUT_WH_YOLO/8))
filter_4 = (df_predictions.loc[:, 'Y_center'] < (Y_center_0 + INPUT_WH_YOLO/8))
filter_final = (filter_1 & filter_2) & (filter_3 & filter_4)

In [248]:
df_predictions_final_2 = df_predictions[filter_final]

In [11]:
def get_neightbours(df_predictions:pd.DataFrame, neightbour:str, index_it:int) -> dict:

    dict_of_neightbours = {} 

    X_center_0 = df_predictions.loc[index_it][0]
    Y_center_0 = df_predictions.loc[index_it][1]
    Width_0 = df_predictions.loc[index_it][2]
    height_0 = df_predictions.loc[index_it][3]
    threshold_x_0 = 0.8 * Width_0
    threshold_y_0 = 0.8 * height_0

    INPUT_WH_YOLO = 640
    filter_1 = (df_predictions.loc[:, 'X_center'] < (X_center_0 + INPUT_WH_YOLO/8))
    filter_2 = (df_predictions.loc[:, 'X_center'] > (X_center_0 - INPUT_WH_YOLO/8))
    filter_3 = (df_predictions.loc[:, 'Y_center'] > (Y_center_0 - INPUT_WH_YOLO/8))
    filter_4 = (df_predictions.loc[:, 'Y_center'] < (Y_center_0 + INPUT_WH_YOLO/8))
    filter_final = (filter_1 & filter_2) & (filter_3 & filter_4)

    df_predictions_final_2 = df_predictions[filter_final]

    for index_bb in df_predictions_final_2.index:

        list_of_neightbours_l = []
        list_of_neightbours_r = []
        list_of_neightbours_h = [] 
        list_of_alones_l = []
        list_of_alones_r = []
        list_of_alones_h=[]

        X_center_neightbour = df_predictions.loc[index_bb][0]
        Y_center_neightbour = df_predictions.loc[index_bb][1]
        a = 4
        k = 1.2

        ### Neightbor Left
        if neightbour == 'left':
            
            x_min_l = X_center_0 - Width_0 - (a*threshold_x_0)
            x_max_l = X_center_0 - Width_0 + (k*threshold_x_0)

            y_min_l = Y_center_0 - (k*threshold_y_0)
            y_max_l = Y_center_0 + (k*threshold_y_0)

            if (x_min_l < X_center_neightbour < x_max_l and y_min_l < Y_center_neightbour < y_max_l )  :
                list_of_neightbours_l.append([index_bb])
                try:
                    dict_of_neightbours[str(index_it)+'_l'].append(index_bb)
                except KeyError:
                    dict_of_neightbours[str(index_it)+'_l']= []
                    dict_of_neightbours[str(index_it)+'_l'].append(index_bb)
            else:
                list_of_alones_l.append([index_bb])

        ### Neightbor Right
        elif neightbour == 'right':

            x_min_r = X_center_0 + Width_0 - (k*threshold_x_0)
            x_max_r = X_center_0 + Width_0 + (a*threshold_x_0)

            y_min_r = Y_center_0 - (k*threshold_y_0)
            y_max_r = Y_center_0 + (k*threshold_y_0)

            if (x_min_r < X_center_neightbour < x_max_r and y_min_r < Y_center_neightbour < y_max_r ):
                list_of_neightbours_r.append([index_bb])
                try:
                    dict_of_neightbours[str(index_it)+'_r'].append(index_bb)
                except KeyError:
                    dict_of_neightbours[str(index_it)+'_r']= []
                    dict_of_neightbours[str(index_it)+'_r'].append(index_bb)
            else:
                list_of_alones_r.append([index_it, index_bb])

        ### Neightbor High
        else:
            x_min_h = X_center_0 - (k*threshold_x_0)
            x_max_h = X_center_0 + (2*k*threshold_x_0)

            y_min_h = Y_center_0 + height_0 - (0.8*threshold_y_0)
            y_max_h = Y_center_0 + height_0 + (2*a*threshold_y_0)

            if (x_min_h < X_center_neightbour < x_max_h and y_min_h < Y_center_neightbour < y_max_h ):
                list_of_neightbours_h.append([index_bb])
                try:
                    dict_of_neightbours[str(index_it)+'_h'].append(index_bb)
                except KeyError:
                    dict_of_neightbours[str(index_it)+'_h']= []
                    dict_of_neightbours[str(index_it)+'_h'].append(index_bb)
            else:
                list_of_alones_h.append([index_it, index_bb])

    return dict_of_neightbours

In [12]:
pool = multiprocessing.Pool()
neightbour = 'left'
func = partial(get_neightbours, df_predictions, neightbour)
dict_of_neightbours_left = pool.map(func, list(df_predictions.index))
pool.close()
pool.join()

dict_of_neightbours_left = list(filter(None, dict_of_neightbours_left))

pool = multiprocessing.Pool()
neightbour = 'right'
func = partial(get_neightbours, df_predictions, neightbour)
dict_of_neightbours_right = pool.map(func, list(df_predictions.index))
pool.close()
pool.join()

dict_of_neightbours_right = list(filter(None, dict_of_neightbours_right))

pool = multiprocessing.Pool()
neightbour = 'up'
func = partial(get_neightbours, df_predictions, neightbour)
dict_of_neightbours_up = pool.map(func, list(df_predictions.index))
pool.close()
pool.join()

dict_of_neightbours_up = list(filter(None, dict_of_neightbours_up))

In [16]:
h_image, w_image = image.shape[0:2]

In [ ]:
dict_of_neightbours_left
dict_of_neightbours_right
dict_of_neightbours_up

In [13]:
list2d = [dict_of_neightbours_left, dict_of_neightbours_right, dict_of_neightbours_up]
merged = list(itertools.chain.from_iterable(list2d))

In [32]:
def search_voids_bb_neightbours(df_predictions: pd.DataFrame, list_of_dicts: list, h_image:int, w_image:int) -> list:
    """_summary_

    Args:
        df_predictions (pd.DataFrame): _description_
        list_of_dicts (list): _description_
        h_image (int): _description_
        w_image (int): _description_
        img_path (_type_): _description_

    Returns:
        list: _description_
    """
    
    list_of_voids = []

    k = 0
    z = 0
    void_number = 0
    # iterate over all dicts in list
    for dicts in list_of_dicts:
        
        # iterate over key and value pairs of dict (index_a = key // index_b = value pair)
        for index_a, index_b in dicts.items():
            
            if index_a[-1] == 'l':
                k = -1
            elif index_a[-1] == 'r':
                k = 1

            index_a = index_a[0:-2]

            #h_image, w_image = image.shape[0:2] # limits of the image
            w_index_a = df_predictions.loc[int(index_a)][2]
            h_index_a = df_predictions.loc[int(index_a)][3]
            
            # Virtual bounding box to evaluate from neightbours 
            xA1 = df_predictions.loc[int(index_a)][0] - df_predictions.loc[int(index_a)][2]/2 + (k * w_index_a) # Para izquierda: (-1) / Para derecha: 1 / Para arriba: 0
            yA1 = df_predictions.loc[int(index_a)][1] - df_predictions.loc[int(index_a)][3]/2 
            xA2 = df_predictions.loc[int(index_a)][0] + df_predictions.loc[int(index_a)][2]/2 + (k * w_index_a) # Para izquierda: (-1) / Para derecha: 1 / Para arriba: 0
            yA2 = df_predictions.loc[int(index_a)][1] + df_predictions.loc[int(index_a)][3]/2 
            boxA = [xA1, yA1, xA2, yA2]

            X_center_A = df_predictions.loc[int(index_a)][0] - k * df_predictions.loc[int(index_a)][2]  # Left X_center - Width  // Right X_center + Width
            Y_center_A = df_predictions.loc[int(index_a)][1]  - k * df_predictions.loc[int(index_a)][3]  # Left Y_center - Width // Right Y_center + Width

            # Limits of the image
            if 0 < xA1 < w_image and 0 < xA2 < w_image and 0 < yA1 < h_image and  0 < yA2 < h_image:
                    trigger = True
                    

                    # Iterate over each neightbour: neightbour vs virtual bounding box
                    for item in index_b:
                        
                        first_list = []
                        xB1 = df_predictions.loc[item][0] - df_predictions.loc[item][2]/2
                        yB1 = df_predictions.loc[item][1] - df_predictions.loc[item][3]/2
                        xB2 = df_predictions.loc[item][0] + df_predictions.loc[item][2]/2
                        yB2 = df_predictions.loc[item][1] + df_predictions.loc[item][3]/2
                        boxB = [xB1, yB1,xB2, yB2]

                        xA = max(boxA[0], boxB[0])
                        yA = max(boxA[1], boxB[1])
                        xB = min(boxA[2], boxB[2])
                        yB = min(boxA[3], boxB[3])

                        interArea = (xB - xA) * (yB - yA)

                        boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
                        boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])

                        iou = interArea / float(boxAArea + boxBArea - interArea)
                        trigger = trigger and (iou < 0.1)

                    if trigger == False:
                        pass
                    else:
                        print(f'A la izquierda de {index_a} hay espacio vacio')
                        void_number += 1
                        void_text = f'{index_a} void #{void_number}'
                        first_list.append(index_a)
                        first_list.append(void_text)
                        first_list.append(xA1)
                        first_list.append(yA1)
                        first_list.append(xA2)
                        first_list.append(yA2)
                        first_list.append(w_index_a)
                        first_list.append(h_index_a)
                        list_of_voids.append(first_list)

    return list_of_voids

In [33]:
list_of_voids = search_voids_bb_neightbours(df_predictions=df_predictions, list_of_dicts=merged, h_image=h_image, w_image=w_image)

A la izquierda de 393 hay espacio vacio
A la izquierda de 391 hay espacio vacio
A la izquierda de 420 hay espacio vacio
A la izquierda de 768 hay espacio vacio
A la izquierda de 89 hay espacio vacio
A la izquierda de 561 hay espacio vacio
A la izquierda de 562 hay espacio vacio
A la izquierda de 943 hay espacio vacio
A la izquierda de 965 hay espacio vacio
A la izquierda de 600 hay espacio vacio


In [36]:
df_voids = pd.DataFrame(list_of_voids, columns=['Neightbour','Label','x1','y1','x2','y2', 'Width','Height'])

In [37]:
df_voids

,Neightbour,Label,x1,y1,x2,y2,Width,Height
0,393,393 void #1,1575.464939,2290.024667,1770.959368,2357.109402,195.494429,67.084735
1,391,391 void #2,1449.175397,2291.150212,1622.804498,2356.895906,173.629102,65.745694
2,420,420 void #3,1259.791166,2424.219517,1455.223125,2511.737850,195.431960,87.518334
3,768,768 void #4,1090.320600,2728.377701,1292.464675,2806.587802,202.144075,78.210100
4,89,89 void #5,1766.191532,2258.931440,1916.023254,2394.208763,149.831721,135.277323
5,561,561 void #6,587.345615,1382.521900,696.249605,1750.651769,108.903990,368.129869
6,562,562 void #7,665.838094,1388.161055,766.640690,1749.020958,100.802596,360.859902
7,943,943 void #8,1926.983928,2240.429894,2059.756581,2385.724574,132.772653,145.294680
8,965,965 void #9,2049.975786,2381.535893,2157.981736,2523.983962,108.005949,142.448070
9,600,600 void #10,910.415169,1465.027548,1017.968460,1765.828678,107.553291,300.801130


In [38]:
#image_path_bb = os.path.join(folder_path_final, filename)

img_path = '/home/cremerf/FinalProject/scripts/test_neightbours2.jpg'
# load the image
image = cv2.imread(img_path)

# loop over indexes and for each index plot the rectangles
#for idx in df_voids.index:

# get the coordinates for each index/rectangle
for i in df_voids.index:
    x1 = int(df_voids.loc[i][2]) 
    y1 = int(df_voids.loc[i][3]) 
    x2 = int(df_voids.loc[i][4])  
    y2 = int(df_voids.loc[i][5])


    # # Window name in which image is displayed
    window_name = 'Object'
    # represents the top left corner of rectangle
    start_point=(x1, y1)

    # represents the top right corner of rectangle
    end_point=(x2,y2)

    # # Blue color in BGR
    color = (0, 0, 255)

    # # Line thickness of 2 px
    thickness = 5

    # plot the rectangle over the image
    image = cv2.rectangle(image, start_point, end_point, color, thickness)

# save the img
cv2.imwrite(filename='test_voids9.jpg', img=image)

True